In [17]:
import json
with open('AF_4.json') as user_file:
  file_contents = user_file.read()
parsed_json = json.loads(file_contents)

In [18]:
class ArgumentationFramework:
    def __init__(self, attacks):
        self.attacks = attacks  # List of tuples representing attacks

    def get_attackers(self, argument):
        """ Return a list of arguments that attack the given argument """
        return [attacker for attacker, attacked in self.attacks if attacked == argument]

class DiscussionGame:
    def __init__(self, framework, claimed_argument):
        self.framework = framework
        self.claimed_argument = claimed_argument
        self.IN = set([claimed_argument])
        self.OUT = set()
        self.UNDEC = set(self.framework.get_attackers(claimed_argument))

    def proponent_turn(self, last_out_argument):
        # Proponent selects an argument from attackers of last_out_argument, excluding IN
        potential_attackers = set(self.framework.get_attackers(last_out_argument))
        # print(f"potential_attackers: {potential_attackers}")
        # print(f"IN: {self.IN}")
        # print(f"OUT: {self.OUT}")
        if not potential_attackers:
            return "Opponent wins", True
        
        potential_attackers = set(self.framework.get_attackers(last_out_argument)) - self.OUT

        chosen_argument = potential_attackers.pop()
        self.IN.add(chosen_argument)
        print(f"Proponent chooses argument: {chosen_argument}")

        # Update UNDEC with attackers of the chosen argument, excluding those in OUT
        new_attackers = set(self.framework.get_attackers(chosen_argument)) - self.OUT
        self.UNDEC.update(new_attackers)

        return None, False

    def opponent_turn(self):
        # If UNDEC is empty, proponent wins
        if not self.UNDEC:
            return "Proponent wins", True

        print(f"Opponent, choose an argument to attack {self.UNDEC}: ")
        chosen_argument = input(f"Opponent, choose an argument to attack {self.UNDEC}: ")

        # Check if the chosen argument is valid
        if chosen_argument not in self.UNDEC:
            print("Invalid choice. Try again.")
            return None, False

        print(f"Opponent chooses argument: {chosen_argument}")
        # Move chosen argument from UNDEC to OUT
        self.UNDEC.remove(chosen_argument)
        self.OUT.add(chosen_argument)

        return chosen_argument, False

    def play_game(self):
        last_out_argument = None
        while True:
            result, game_over = self.opponent_turn()
            if game_over:
                print(result)
                break

            last_out_argument, game_over = result, False

            result, game_over = self.proponent_turn(last_out_argument)
            if game_over:
                print(result)
                break

# Example Usage
# attacks = [('0', '1'), ('1', '0'), ('2', '0'), ('3', '0'), ('3', '1'), ('4', '3'), ('5', '0'), ('5', '1'), ('6', '5')]
attacks = parsed_json['Attack Relations']
framework = ArgumentationFramework(attacks)
game = DiscussionGame(framework, '4')
game.play_game()

Opponent, choose an argument to attack {'3'}: 
Opponent chooses argument: 3
Proponent chooses argument: 5
Opponent, choose an argument to attack {'4'}: 
Invalid choice. Try again.
Opponent wins
